## <center id="c1"><h2> 🦜🔗 `Structured Output` в `LangChain` - получи ответ от LLM в нужном формате! </h2>

<img src="../images/so.png" align="right" width="400" height="300" style="border-radius: 0.75rem;" alt="LangChain Image" />

### Оглавление ноутбука:

 * <a href="#c1"> Основные компоненты SO  </a>
 * <a href="#c2"> 🧾 Response schema - говорим модели про свои хотелки! </a> 
 * <a href="#look1"> 🧶 Метод with_structured_output() - привязка схемы! </a>
 * <a href="#check1"> 🤝 Pydantic схема = формат + валидация! </a>
 * <a href="#check2"> ✌️ Выбираем между двумя схемами! </a>
 * <a href="#look2"> 🔖 JSON mode</a>
 * <a href="#6">🧸 Выводы и заключения</a>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Существуют сценарии, в которых нам нужно получить ответ от модели в структурированном формате. Например, мы можем захотеть сохранить выходные данные модели в базе данных и убедиться, что выходные данные соответствуют схеме базы данных. 

Эта необходимость обусловливает концепцию структурированного ответа (`Structured output`), при которой моделям может быть дано указание использовать определенную структуру выходных данных. <br>

Не все модели поддерживают такой формат ответа - перед использованием, смотрите [список](https://python.langchain.com/docs/integrations/chat/#featured-providers).


### **Основные компоненты SO (смотрите рисунок выше):**

<div class="alert alert-custom" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

1. **Определение схемы (Response schema):** Структура выходных данных представляется в виде схемы, которая может быть определена несколькими способами.
2. **Возврат структурированного ответа:** модели предоставляется эта схема, и дается указание возвращать выходные данные, соответствующие ей.

<div style="background-color:#fff0ff; padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Можно проиллюстрировать концепцию таким псевдокодом:
```python
# Определяем схему
schema = {"foo": "bar"}

# Привязываем схему к модели
model_with_structure = model.with_structured_output(schema)

# Делаем запрос к модели, с привязанной схемой
structured_output = model_with_structure.invoke(user_input)    
```

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Помните как мы в первом ноутбуке по `LangChain` разбирали ответ модели с помощью `Output Parser`? <br>Давайте теперь попробуем решить ту же задачу с помощью `Structured Output`.

# <center> 🔑 Вводим ключ 

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install langchain langchain-openai openai -q

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

# course_api_key= "Введите API-ключ полученный в боте"
course_api_key = getpass(prompt='Введите API-ключ полученный в боте:')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://api.vsellm.ru/")

Введите API-ключ полученный в боте: ········


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Допустим у нас есть база отзывов покупателей, мы хотим подать отзыв на вход модели, а на выходе получить готовый Python словарь(как представлено выше) для дальнейшего использования.

In [3]:
customer_review = """
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.
"""

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Давайте определим то, как мы хотим, чтобы выглядели выходные данные из LLM:

In [5]:
{
  "product": "фен для волос",
  "gift": False,
  "delivery_days": 5,
  "price_value": "подходящая"
}

{'product': 'фен для волос',
 'gift': False,
 'delivery_days': 5,
 'price_value': 'подходящая'}

# <center id="c2"> 🧾 `Response schema` - говорим модели про свои хотелки!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Схема может быть указана как:
* класс `TypedDict`
* схема `JSON`
* класс `Pydantic`

Если используется `TypedDict` или схема `JSON`, то модель вернет словарь, а если используется класс `Pydantic`, то будет возвращен объект `Pydantic`.<br>
    

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Как правило, в схеме любого типа должны быть основные элементы:
* Описание класса (может быть пропущено)
* Описание каждого поля (может быть пропущено)
* Тип поля (строки, числа, списки и.т.п)
* Значение по умолчанию (может быть пропущено)

Чем точнее вы опишите схему, тем модели проще будет выдавать результат в нужном формате.

In [6]:
from typing_extensions import Annotated, TypedDict

class Customer_Review(TypedDict):
    """Отзыв покупателя""" # Описание класса

    product: Annotated[str, ..., "Название товара"] # тип, значение по умолчанию, описание

    # Как ещё можно было описать поле product:
    
    # product: str                      # нет значения по умолчанию, нет описания
    # product: Annotated[Optional[str]] # не обязательное поле
    # product: Annotated[str, ...]      # нет значения по умолчанию, нет описания
    # product: Annotated[str, "не определен"]  # есть значение по умолчанию, нет описания

    gift: Annotated[bool, ..., "Является ли подарком"]
    delivery_days: Annotated[int, ..., "Количество дней доставки"]
    price_value: Annotated[str, "отсутствует", "Описание цены"]

# <center id="c2"> 🧶 Метод `with_structured_output()` - привязка схемы!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Метод `with_structured_output()` использует схему в качестве входных данных. <br> Метод возвращает моделеподобный `Runnable`, за исключением того, что вместо вывода строк или сообщений он выводит объекты, соответствующие заданной схеме.<br>
    

In [8]:
# Привязываем схему к llm
llm_with_so = llm.with_structured_output(Customer_Review)

response = llm_with_so.invoke(customer_review)
response

{'product': 'Фен для волос',
 'gift': True,
 'delivery_days': 2,
 'price_value': 'Чуть дороже, чем аналоги, но с хорошими функциями.'}

In [9]:
type(response)

dict

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Модель выдала ответ в желаемом формате и нужного типа!

Рассмотрим пример с `JSON` схемой:

In [12]:
json_schema = {
    "title": "Customer_review",        # название схемы
    "description": "Отзыв покупателя", # описание
    "type": "object",
    "properties": {                    # описание полей
        "product": {"type": "string", "description": "Название продукта"},
        "gift": {"type": "boolean", "description": "Является ли подарком"},
        "delivery_days": {"type": "integer", "description": "Количество дней доставки"},
        "price_value": {"type": "string", "description": "Описание цены"}
    },
    "required": ["product", "gift", "delivery_days", "price_value"] # список обязательных полей
}

structured_llm = llm.with_structured_output(json_schema)
response = structured_llm.invoke(customer_review)
response, type(response)

({'product': 'Фен для волос',
  'gift': True,
  'delivery_days': 2,
  'price_value': 'немного дороже, чем другие, но дополнительные функции того стоят'},
 dict)

# <center id="c2"> 🤝 `Pydantic` схема = формат + валидация!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Ключевым преимуществом использования `Pydantic` схемы является то, что выходные данные, сгенерированные с помощью модели, будут проверены. <br>
`Pydantic` выдаст сообщение об ошибке, если какие-либо обязательные поля отсутствуют или имеют неправильный тип.
    

In [13]:
from pydantic import BaseModel, Field
from typing import Optional

class Customer_Review(BaseModel):
    """Отзыв покупателя""" # Описание класса

    product: str = Field(..., description="Название продукта")
    gift: bool = Field(..., description="Является ли подарком")
    delivery_days: int = Field(..., description="Количество дней доставки")
    price_value: str = Field(default='отсутствует', description="Описание цены")
    
    # Если бы поле price_value было не обязательным, можно было бы указать его так:
    # price_value: Optional[str] = Field(default='отсутствует', description="Описание цены")

In [14]:
structured_llm = llm.with_structured_output(Customer_Review)
response = structured_llm.invoke(customer_review)
response

Customer_Review(product='Фен для волос', gift=True, delivery_days=2, price_value='немного дороже, чем другие')

In [15]:
type(response)

__main__.Customer_Review

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Получили результат в виде объекта `Pydantic`.

# <center id="c2"> ✌️ Выбираем между двумя схемами!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Например, мы собираем отзывы о товарах через чат-бота и пользователи могут с ним просто общаться. Как привязать к модели 2 или более схем и осуществлять выбор? 🤔

Рассмотрим ниже на примере:
* Возьмём нашу предыдущую схему для разбора отзывов
* Создадим ещё одну схему для общих вопросов
* Создадим финальную схему, где объединим их вместе через `Union`
* И именно её привяжем к модели

In [16]:
from typing import Union


class Customer_Review(TypedDict):
    """Отзыв покупателя""" # Описание класса

    product: Annotated[str, ..., "Название товара"] # тип, значение по умолчанию, описание
    gift: Annotated[bool, ..., "Является ли подарком"]
    delivery_days: Annotated[int, ..., "Количество дней доставки"]
    price_value: Annotated[str, "отсутствует", "Описание цены"]


class ConversationalResponse(TypedDict):
    """Отвечай в диалоговом формате. Будь дружелюбен и полезен"""

    response: Annotated[str, ..., "Ответ на вопрос пользователя"]


class FinalResponse(TypedDict):
    final_output: Union[Customer_Review, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

structured_llm.invoke(customer_review)

{'final_output': {'Customer_Review': {'product': 'Фен для волос',
   'gift': True,
   'delivery_days': 2,
   'price_value': 'Дороже, чем другие, но дополнительные функции того стоят.'}}}

In [17]:
structured_llm.invoke('Как дела?')

{'final_output': {'response': 'Привет! У меня всё хорошо, спасибо! Как твои дела?'}}

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Модель в зависимости от запроса использует ту или иную схему.

# <center id="c2"> 🔖 JSON mode

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Некоторые модели, в частности от `OpenAI`, поддерживают ответ в `JSON mode`.<br>
Т.е. отвечают сразу в виде словаря, без привязки схемы. Но такой формат пока поддерживают небольшое число вендоров - [список](https://python.langchain.com/docs/integrations/chat/).

In [22]:
# указываем, что хотим получить ответ в формате JSON
json_model = llm.with_structured_output(method="json_mode")

response = json_model.invoke(
    "Верни JSON объект с ключом 'random_llms' и значением из 3 рандомных названий LLM.")
response

{'random_llms': ['GPT-3', 'BERT', 'T5']}

In [23]:
type(response)

dict

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Модель по запросу без схемы выдаёт словарь в нужном формате.

# <center> 🧸 Выводы и заключения

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

В этом уроке разобрались как добиться от модели структурированного ответа `Structured Output`.

- Такой ответ позволяет быстро и бесшовно использовать результаты работы модели, и уменьшать количество галлюцинаций
- Качество такого ответа будет зависеть от грамотно составленной `Response schema`
- Провалидировать ответ сразу можно через `Pydantic` схему
- В описание полей можно добавлять инструкции, для разных случаев или желаемого поведения